## 1. 문제정의 - 텍스트를 분류하는 모델 만들기
   - 목적: 영화 리뷰를 보고 긍정(좋다)인지 부정(싫다)인지 자동으로 판닪는 모델을 만든다.
   - 입력: 영화 리뷰 텍스트(예: "This movie was great!")
   - 출력: 0(부정) 또는 1(긍정)

## 2. 데이터 준비 - 영화 리뷰 데이터를 가져오기
   - 사용하는 데이터셋: IMDB 영화 리뷰 데이터셋

## 3. 모델준비 - 사전학습된 똑똑한 모델 불러오기
   - 처음부터 모델을 만드는 게 아니라, 이미 영어를 어느 정도 학습해둔 모델(DistilBERT)을 가져온다.
   - 이 모델에 "긍정과 부정 분류"라는 새로운 임무를 학인

## 4. 모델훈련 - 데이터로 학습시키기
   - 앞에서 준비한 텍스트(숫자 형태)와 라벨(긍정/부정)을 가지고 모델을 학습한다.
   - 학습은 30번 반복해서 진행된다.(30 에폭)

## 5. 예측하기 - 새로운 문장에 대해 모델이 판단하도록 하

In [1]:
!pip install -q transformers datasets

In [ ]:
!pip install -U 'datasets<=2.18.0' 'fsspec<=2023.6.0'

In [3]:
## 데이터 로드
from datasets import load_dataset

dataset = load_dataset("imdb", split="train[:50]").train_test_split(test_size=0.2)

# IMDB 영화 리뷰 데이터셋 중 50개 샘플만 가져와서 학습용/평가용으로 8:2 비율

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
sample = dataset["train"][5]
print(f"리뷰 내용:\n{sample['text']}\n")
print(f"라벨 (0=부정, 1=긍정): {sample['label']}")

리뷰 내용:
I have this film out of the library right now and I haven't finished watching it. It is so bad I am in disbelief. Audrey Hepburn had totally lost her talent by then, although she'd pretty much finished with it in 'Robin and Marian.' This is the worst thing about this appallingly stupid film. It's really only of interest because it was her last feature film and because of the Dorothy Stratten appearance just prior to her homicide.<br /><br />There is nothing but idiocy between Gazzara and his cronies. Little signals and little bows and nods to real screwball comedy of which this is the faintest, palest shadow.<br /><br />Who could believe that there are even some of the same Manhattan environs that Hepburn inhabited so magically and even mythically in 'Breakfast at Tiffany's' twenty years earlier? The soundtrack of old Sinatra songs and the Gershwin song from which the title is taken is too loud and obvious--you sure don't have to wait for the credits to find out that something w

In [6]:
# 모델 및 토크나이저 준비
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# 토크나이저: 텍스트를 숫자 토큰으로 바꾸는 역할
# "나는 배가 고프다." -> [101, 1432, 2234, 1672, 102] <- 이런식으로 숫자로 바꿔줘야 모델이 이해 가능

model =AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
# distilbert-base-uncased: BERT를 경량화한 모델(빠르고 정확도가 괜찮다.)
# 메인 포인트: 사전학습된 모델이므로, 적은 데이터로도 빠르게 학습이 가능

# 데이터 전처리 (텍스트를 숫자 토큰으로 변환)
def tokenize(batch):
  return tokenizer(batch['text'], truncation=True, padding=True)

# 포인트: tokenizer함수로 모든 리뷰 텍스트를 숫자 토큰으로 바꾼다.
# truncation =True 문장이 길면 자른다
# padding=True 짧은 문장은 0으로 채워 길이를 맞춘다.

dataset = dataset.map(tokenize, batched=True)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [8]:
# 훈련 설정 및 Trainer 구성
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score

# 정확도 계산 함수
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = logits.argmax(axis=-1)
  acc = accuracy_score(labels, predictions)
  return {"accuracy": acc}

args = TrainingArguments(
    output_dir ="test", # 모델이 저장될 경로
    per_device_train_batch_size=8, # 학습시 배치 크기(GPU 하나당 8개씩 처리)
    num_train_epochs=15, # 에폭 수 (데이터셋을 15번 반복해서 학습)
    report_to = "none", # wandb, tensorboard 등 외부 로깅 툴 비활성화
      logging_steps=1 # 몇 step마다 로그 출력할지
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset = dataset['train'], #학습용 데이터셋
    eval_dataset = dataset['test'], #검증용 데이터셋
    compute_metrics = compute_metrics # 정확도 계산 함수
)

In [9]:
# 파인튜닝 실행(모델 학습)
trainer.train()

Step,Training Loss
1,0.780800
2,0.595400
3,0.484500
4,0.355000
5,0.235300
6,0.177500
7,0.128100
8,0.083300
9,0.060800
10,0.044700


TrainOutput(global_step=75, training_loss=0.04279766480904072, metrics={'train_runtime': 41.6629, 'train_samples_per_second': 14.401, 'train_steps_per_second': 1.8, 'total_flos': 79480439193600.0, 'train_loss': 0.04279766480904072, 'epoch': 15.0})

In [10]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.0009463358437642455, 'eval_accuracy': 1.0, 'eval_runtime': 0.2413, 'eval_samples_per_second': 41.447, 'eval_steps_per_second': 8.289, 'epoch': 15.0}


In [11]:
# 학습된 모델로 실제 예측 수행
text = "It's a movie list that's not worth watching again."
inputs = tokenizer(text, return_tensors="pt").to("cuda")
# 포인트: 새 문장을 토큰화하고 나서, gpus로 보내줌

output = model(**inputs)
label = output.logits.argmax(-1).item()
# 포인트: 모델 출력(logits)은 [부정 점수, 긍정 점수] 형태의 벡터
# argmax(-1): 가장 높은 점수의 인덱스 (0 또는 1)를 예측값으로 사용

print("긍정" if label==1 else "부정")
# 최종 결과 출력: 긍정인지 부정인지 해석해서 보여

부정
